In [6]:
# %%capture
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
# %pip install huggingface_hub[hf_xet]

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
  Using cached http://mirrors.aliyun.com/pypi/packages/72/27/ec6ee3408e09e01ab05db07af5a97dc76db7bc18824cf5f5dbc98e1e08a4/bitsandbytes-0.46.0-py3-none-manylinux_2_24_x86_64.whl (67.0 MB)
  Using cached http://mirrors.aliyun.com/pypi/packages/0a/7c/0a5b3aee977596459ec45be2220370fde8e017f651fecc40522fd478cb1e/torch-2.7.1-cp310-cp310-manylinux_2_28_x86_64.whl (821.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:0000:0100:01
  Using cached http://mirrors.aliyun.com/pypi/packages/8f/16/73727675941ab8e6ffd86ca3a4b7b47065edcca7a997920b831f8147c99d/nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_

In [ ]:
# from huggingface_hub import login
# import os

# hf_token = os.environ.get("HF_TOKEN")
# login(hf_token)

In [2]:
import torch
torch.cuda.is_available()

True

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [4]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=False,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

PackageNotFoundError: No package metadata was found for bitsandbytes

In [8]:
model_dir = "../models/qwen3-0.6b"

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    # quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

In [9]:
!nvidia-smi

Tue Jul  1 19:56:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:98:00.0 Off |                  Off |
| 30%   31C    P2             53W /  450W |    1847MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:

def tokenize(example):
    prompt = f"""You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0:p Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
{example['src']}

### MT sentence:
{example['mt']}

"""
    completion = f""" {example['zmean']:.4f}"""

    prompt_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
    completion_ids = tokenizer(completion, add_special_tokens=False)["input_ids"] + [tokenizer.eos_token_id]
    input_ids = prompt_ids + completion_ids

    labels = [-100] * len(prompt_ids) + completion_ids
    attention_mask = [1] * len(input_ids)

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask
    }

In [11]:
trainpath = "../data/toy_train.tsv"
devpath = "../data/toy_dev.tsv"
testpath = "../data/toy_test.tsv"

In [13]:
import pandas as pd
from datasets import Dataset
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

traindf = pd.read_csv(trainpath, sep="\t")
devdf = pd.read_csv(devpath, sep="\t")
testdf = pd.read_csv(testpath, sep="\t")

train_dataset = Dataset.from_pandas(traindf)
dev_dataset = Dataset.from_pandas(devdf)
test_dataset = Dataset.from_pandas(testdf)

tokenized_train = train_dataset.map(
    tokenize,
    batched=False,
    remove_columns=train_dataset.column_names,
)

tokenized_dev = dev_dataset.map(
    tokenize,
    batched=False,
    remove_columns=dev_dataset.column_names,
)

tokenized_test = test_dataset.map(
    tokenize,
    batched=False,
    remove_columns=test_dataset.column_names,
)
print(tokenized_train[16])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

{'input_ids': [2610, 525, 264, 2745, 49823, 14468, 16460, 6203, 13, 4615, 3383, 374, 311, 7023, 279, 4271, 5456, 369, 14468, 13530, 624, 785, 4271, 5456, 374, 264, 1372, 1948, 481, 17, 13, 15, 323, 220, 17, 13, 15, 323, 1410, 4399, 1119, 279, 2701, 11059, 510, 3326, 5503, 14243, 510, 12, 17, 13, 15, 311, 481, 16, 13, 15, 25, 44673, 476, 1602, 7852, 14468, 448, 7290, 37564, 476, 15386, 5975, 624, 12, 16, 13, 15, 311, 220, 15, 13, 15, 44389, 2988, 672, 291, 14468, 714, 48739, 624, 15, 13, 15, 311, 220, 16, 13, 15, 25, 7684, 14468, 304, 4586, 624, 16, 13, 15, 311, 220, 17, 13, 15, 25, 36766, 476, 68516, 14468, 624, 785, 8674, 2309, 315, 279, 12205, 304, 419, 2088, 14807, 26447, 11799, 304, 14468, 4271, 382, 14374, 8748, 11652, 510, 1109, 6951, 287, 68834, 47524, 369, 806, 5786, 450, 6767, 11, 1340, 16426, 1413, 1435, 448, 806, 1828, 24004, 47524, 11, 323, 10901, 1435, 827, 904, 3073, 323, 3143, 4545, 382, 14374, 19087, 11652, 510, 101928, 113736, 9370, 99695, 22243, 41175, 4891, 98, 117, 

In [14]:
original_example = train_dataset[16]
prompt = f"""You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
{original_example['src']}

### MT sentence:
{original_example['mt']}
"""

completion = f" {original_example['zmean']:.4f}"

prompt_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
completion_ids = tokenizer(completion, add_special_tokens=False)["input_ids"] + [tokenizer.eos_token_id]
full_ids = prompt_ids + completion_ids

decoded_prompt = tokenizer.decode(prompt_ids, skip_special_tokens=True)
decoded_completion = tokenizer.decode(completion_ids, skip_special_tokens=True)
decoded_full = tokenizer.decode(full_ids, skip_special_tokens=True)

# 打印
print("\n=== Decoded Prompt ===")
print(decoded_prompt)

print("\n=== Decoded Completion ===")
print(decoded_completion)

print("\n=== Decoded Full Input ===")
print(decoded_full)


=== Decoded Prompt ===
You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
Confronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.

### MT sentence:
面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。


=== Decoded Completion ===
 -0.4941

=== Decoded Full Input ===
You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number b

In [15]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

Model Inference before Finetuning

In [16]:
inference_prompt_style = """You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is an exact number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
{src}

### MT sentence:
{mt}

### Your score:
{score}"""

In [17]:
src_sentence = train_dataset['src'][16]
mt_sentence = train_dataset['mt'][16]
inputs = tokenizer(
    [inference_prompt_style.format(src=src_sentence, mt=mt_sentence, score="") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response)

ImportError: cannot import name '_data_ptr_allocated' from 'torch.distributed.utils' (/root/miniconda3/lib/python3.10/site-packages/torch/distributed/utils.py)

In [26]:
import re
def extract_score_from_response(text, default_score=-2.0):
    # 定位 "### Your score:" 后最近的一个浮点数
    pattern = r"### Your score:\s*.*?(-?\d+\.\d+|-?\d+)"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return float(match.group(1))
    else:
        return default_score
print(type(response[0]))
print(response[0])
score = extract_score_from_response(response[0])
print(score)

<class 'str'>
You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
Many productions were musicals, comedies, melodramas or westerns.

### MT sentence:
Viele Produktionen waren Musicals, Komödien, Melodramen oder Western.

### Quality score:

ාන්නේ විසින් ආදුවිය නිස්තා වීමට දහම විස්තිමය ප්‍රද්ධ ලේකි.

### Quality score:
-0.0877

-2.0


Start Training

In [20]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

# Training Arguments
training_args = SFTConfig(
    output_dir="~/autodl-fs/my_project/output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=2,
    logging_steps=10,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    completion_only_loss=True,
    group_by_length=True,
    report_to="none"
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator,
    eval_dataset=tokenized_dev,
)

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/125 [00:00<?, ? examples/s]

/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


AttributeError: module 'torch' has no attribute 'xpu'

In [16]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

Step,Training Loss
200,1.466700
400,1.358000
600,1.032200
800,0.776800
1000,0.728100


TrainOutput(global_step=1000, training_loss=1.0723645782470703, metrics={'train_runtime': 2050.1422, 'train_samples_per_second': 0.976, 'train_steps_per_second': 0.488, 'total_flos': 1355499834703872.0, 'train_loss': 1.0723645782470703})

In [17]:
torch.save(model.state_dict(), '/content/drive/MyDrive/my_model.pt')

In [ ]:
predict_output = trainer.predict(train_dataset)

preds = predict_output.predictions  # shape: (num_samples, output_dim)
labels = predict_output.label_ids 
if preds.ndim == 2 and preds.shape[1] == 1:
    preds = preds.squeeze(-1)

# 如果 preds 是 torch.Tensor，需要转numpy
if isinstance(preds, torch.Tensor):
    preds = preds.cpu().numpy()
if isinstance(labels, torch.Tensor):
    labels = labels.cpu().numpy()

# 4. 计算指标
pearson_r, _ = pearsonr(labels, preds)
rmse = mean_squared_error(labels, preds, squared=False)
mae = mean_absolute_error(labels, preds)

print(f"Train set Pearson r: {pearson_r:.4f}")
print(f"Train set RMSE: {rmse:.4f}")
print(f"Train set MAE: {mae:.4f}")

In [21]:
src_sentence = traindf['src'][16]
mt_sentence = traindf['mt'][16]
inputs = tokenizer(
    [inference_prompt_style.format(src=src_sentence, mt=mt_sentence, score="") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
score = extract_score_from_response(response[0])
print(score)
print(response[0].split("### Response:"))

-1.4782
['You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.\nThe quality score is an exact number between -2.0 and 2.0 and could fall into the following categories:\nScoring Criteria:\n-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.\n-1.0 to 0.0: Flawed translation but understandable.\n0.0 to 1.0: Good translation in general.\n1.0 to 2.0: Excellent or flawless translation.\nThe relative position of the scores in this range indicates subtle differences in translation quality.\n\n### Source sentence:\nConfronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.\n\n### MT sentence:\n面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。\n\n### Your score:\nාදී මෙම පිහිස් මත ප්\u200dරත්\u200dයයේ පෙස දෙස දෙස වැඩි ප්\u200dරත්\u200dයයේ පිහුවය විය.\n\n### Quality score:\n-1.4782\n']


In [ ]:
# print(dev_dataset['zmean'][0])
# src_sentence = dev_dataset['src'][0]
# mt_sentence = dev_dataset['mt'][0]
# inputs = tokenizer(
#     [inference_prompt_style.format(src=src_sentence, mt=mt_sentence, score="") + tokenizer.eos_token],
#     return_tensors="pt"
# ).to("cuda")
# outputs = model.generate(
#     input_ids=inputs.input_ids,
#     attention_mask=inputs.attention_mask,
#     max_new_tokens=1200,
#     eos_token_id=tokenizer.eos_token_id,
#     use_cache=True,
# )
# response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# score = extract_score_from_response(response[0])
# print(score)

In [24]:
import numpy as np
from scipy.stats import pearsonr
from tqdm import tqdm


model.eval()
predicted_scores = []
true_scores = []

for sample in tqdm(test_dataset):
  src = sample["src"]
  mt = sample["mt"]
  true_score = float(sample["zmean"])
  prompt = inference_prompt_style.format(src=src, mt=mt, score = "")
  inputs = tokenizer(prompt + tokenizer.eos_token, return_tensors="pt").to(model.device)

  with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=1200,
        eos_token_id=tokenizer.eos_token_id,
        use_cache = True)
  response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  predicted_score = extract_score_from_response(response[0])
  predicted_scores.append(predicted_score)
  true_scores.append(true_score)

pearson_corr, _ = pearsonr(true_scores, predicted_scores)

100%|██████████| 125/125 [14:12<00:00,  6.82s/it]


In [25]:
print(pearson_corr)

0.023705307678204585


In [26]:
print(len(prompt))

762


In [27]:
print(type(prompt))

<class 'str'>


In [28]:
print(prompt[:50])

You are a multilingual translation evaluation expe


In [29]:
print(prompt)

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is an exact number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
ඇය ඔහුට දන්වා එවූයේ සිය සැමියා වූ රජු, නිභුරුරියා මියගොස් ඇති බවයි.

### MT sentence:
She informed him that the king of her husband, the native had died.

### Your score:



In [19]:
new_train_set = Dataset.from_pandas(traindf)
new_train_setje = new_train_set.map(
    formatting_prompts_func,
    batched=True,
)



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [20]:
print(new_train_setje['text'][16])

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
Confronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.

### MT sentence:
面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。

### Quality score:
-0.4941



In [22]:
import re
def remove_gold_score(text):
    # 删除从 "### Quality score:" 到文本结尾的内容
    return re.sub(r"(### Quality score:\s*)(-?\d+(\.\d+)?)(.*)", r"\1", text)
stripped_prompts = [remove_gold_score(t) for t in train_dataset["text"]]

In [23]:
print(stripped_prompts[16])

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
Confronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.

### MT sentence:
面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。

### Quality score:




In [36]:
print(type(stripped_prompts))

<class 'list'>


In [37]:
print(len(stripped_prompts))

1000


In [42]:
print(len(new_train_set))
print(new_train_set[16])

1000
{'index': 1874, 'src': 'Confronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.', 'mt': '面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。', 'zmean': -0.4941006247433139, 'source_file': 'en-zh.tsv', 'root_folder': 'wmt22_fixed', 'file_type': 'zmean', 'global_index': 55874}


In [27]:
import numpy as np
from scipy.stats import pearsonr
from tqdm import tqdm

model.eval()
predicted_scores = []
true_scores = []

for prompt, sample in tqdm(zip(stripped_prompts, new_train_set), total=len(new_train_set)):
    true_score = float(sample["zmean"])
    inputs = tokenizer(prompt + tokenizer.eos_token, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=1200,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    predicted_score = extract_score_from_response(response[0])

    predicted_scores.append(predicted_score)
    true_scores.append(true_score)

# 评估
from scipy.stats import pearsonr
pearson_corr, _ = pearsonr(true_scores, predicted_scores)
print(f"Pearson correlation on training set: {pearson_corr:.4f}")

100%|██████████| 1000/1000 [2:28:57<00:00,  8.94s/it]

Pearson correlation on training set: nan



/tmp/ipython-input-27-3213185660.py:31: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearson_corr, _ = pearsonr(true_scores, predicted_scores)


In [28]:
print(set(true_scores))         # 看看是不是一个元素
print(set(predicted_scores))    # 同上
print(len(true_scores))

{0.8092989049049368, 1.197944436715293, -0.968108984167209, 1.0895254109792205, 1.152311625851567, -0.319318762, 0.1968598850673624, 1.4822063999737782, -0.7493597431336001, -0.692718642, 1.119817375508961, 1.4432122276271002, -1.90719567, 1.086355302437795, -1.7548149770676458, 0.4432220838889799, 8.45431318094125, -2.485258092336451, 0.7107433962198438, -0.6213715510692414, -0.6474679749368836, -0.2360794419564392, 0.0709133640254498, -0.9651099656285989, 0.9460501620131906, 0.5933388360160704, 0.3527437730519095, 0.4428013833584596, 0.3947221872943565, 0.9483706314416808, 0.7054678953623083, 0.643229381423528, 0.0485468697071392, 0.2446832247508253, 0.1787702446814885, 0.2475259729418227, 0.1705219585958702, 0.2375317196779756, 1.4956438867534425, -0.195558514, -0.9164177781238996, -0.8558540319404412, -0.7084912718194002, -1.3536954809046395, 0.3207834698884568, 0.3814397289391078, 0.4784561250943906, -0.1160851246733674, 0.9881728951515012, -0.7003570523504195, -0.7306287845896074

In [30]:


# 2. 检查 prompt 中是否有出现 -2.0
for prompt in train_dataset["text"][:5]:
    print(prompt)

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
Many productions were musicals, comedies, melodramas or westerns.

### MT sentence:
Viele Produktionen waren Musicals, Komödien, Melodramen oder Western.

### Quality score:
0.8093

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
